In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import dedupe
import time

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
gt_train = pd.read_csv("../datasets/ground_truth/GT_train/train.csv")
gt_val = pd.read_csv("../datasets/ground_truth/GT_train/val.csv")
gt_test = pd.read_csv("../datasets/ground_truth/GT_train/test.csv")

In [4]:
df_unificato = pd.read_csv("../datasets/mediated_schema/mediated_schema_normalized.csv", dtype={'id_source_vehicles': 'object'})

/tmp/ipykernel_20415/1061716945.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unificato = pd.read_csv("../datasets/mediated_schema/mediated_schema_normalized.csv", dtype={'id_source_vehicles': 'object'})


In [5]:
# 1. Creiamo la colonna id_unificato combinando le due sorgenti
# Usiamo fillna() perché abbiamo garantito che dove manca uno c'è l'altro
df_unificato['id_unificato'] = (
    df_unificato['id_source_vehicles']
    .fillna(df_unificato['id_source_used_cars'])
)

# 2. Impostiamo l'id_unificato come INDICE del DataFrame
# Questo è il requisito fondamentale per compare.compute()
df_unificato = df_unificato.set_index('id_unificato')

In [6]:
df_unificato.shape

(3255536, 19)

In [7]:
df_unificato.head()

,id_source_vehicles,id_source_used_cars,location,price,year,manufacturer,model,cylinders,fuel_type,mileage,transmission,vin,traction,body_type,main_color,description,latitude,longitude,pubblication_date
id_unificato,,,,,,,,,,,,,,,,,,,
7316814884,7316814884,NaN,auburn,33590.0,2014.0,gmc,sierra 1500 crew cab slt,8 cylinders,gas,57923.0,other,3GTP1VEC4EG551563,NaN,pickup,white,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:18-0500
7316814758,7316814758,NaN,auburn,22590.0,2010.0,chevrolet,silverado 1500,8 cylinders,gas,71229.0,other,1GCSCSE06AZ123805,NaN,pickup,blue,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:08-0500
7316814989,7316814989,NaN,auburn,39590.0,2020.0,chevrolet,silverado 1500 crew,8 cylinders,gas,19160.0,other,3GCPWCED5LG130317,NaN,pickup,red,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:25-0500
7316743432,7316743432,NaN,auburn,30990.0,2017.0,toyota,tundra double cab sr,8 cylinders,gas,41124.0,other,5TFRM5F17HX120972,NaN,pickup,red,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T10:41:31-0500
7316356412,7316356412,NaN,auburn,15000.0,2013.0,ford,f150 xlt,6 cylinders,gas,128000.0,automatic,NaN,rwd,truck,black,2013 f 150 xlt v6 4 door good condition leveli...,32.592,-85.5189,2021-05-03T14:02:03-0500


In [8]:
df_unificato.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3255536 entries, 7316814884 to S2_3000039
Data columns (total 19 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id_source_vehicles   object 
 1   id_source_used_cars  object 
 2   location             object 
 3   price                float64
 4   year                 float64
 5   manufacturer         object 
 6   model                object 
 7   cylinders            object 
 8   fuel_type            object 
 9   mileage              float64
 10  transmission         object 
 11  vin                  object 
 12  traction             object 
 13  body_type            object 
 14  main_color           object 
 15  description          object 
 16  latitude             float64
 17  longitude            float64
 18  pubblication_date    object 
dtypes: float64(5), object(14)
memory usage: 496.8+ MB


In [9]:
# 1. Identifichiamo le colonne per tipologia in base ai tuoi 'fields'
cols_string = ['location', 'manufacturer', 'model', 'cylinders', 
               'description', 'transmission', 'fuel_type', 
               'traction', 'body_type', 'main_color']

cols_numeric = ['price', 'year', 'mileage', 'latitude', 'longitude']

# 2. Pulizia Colonne Testuali (Object)
for col in cols_string:
    # Convertiamo tutto in stringa, ma trasformiamo i 'nan' testuali in stringhe vuote
    # Dedupe 3.0 preferisce stringhe vuote o None per i campi String
    df_unificato[col] = df_unificato[col].astype(str).replace(['nan', 'None', 'NaN'], '')

# 3. Pulizia Colonne Numeriche (float64)
for col in cols_numeric:
    # Per i numeri, Dedupe 3.0 richiede l'oggetto None di Python per i mancanti
    # Questo permette a 'has_missing': True di funzionare correttamente
    df_unificato[col] = df_unificato[col].replace({np.nan: None})

In [ ]:
# --- STEP C: CAMPIONAMENTO STRATIFICATO ---
# 1. ID per Addestramento e Validazione (Dalla tua Ground Truth)
ids_gt_train_val = set(gt_train['id_A']) | set(gt_train['id_B']) | \
                   set(gt_val['id_A']) | set(gt_val['id_B'])

# 2. Campione casuale dal dataset per le statistiche di Blocking (es. 30.000 record)
# Usiamo solo record che NON sono nel Test Set per purismo metodologico
df_no_test = df_unificato.loc[~df_unificato.index.isin(set(gt_test['id_A']) | set(gt_test['id_B']))]
ids_sample = set(df_no_test.sample(n=min(5000, len(df_no_test)), random_state=42).index)

# 3. Unione ID e creazione dizionario
ids_finali = ids_gt_train_val | ids_sample
data_train_only = df_unificato.loc[list(ids_finali), cols_string + cols_numeric].to_dict('index')

# LIBERA RAM: Se il PC è al limite, puoi eliminare df_unificato qui (avendolo salvato su CSV prima)
# del df_unificato
# gc.collect()

print(f"Dizionario creato con {len(data_train_only)} record.")

Dizionario creato con 51279 record.


In [11]:
# Definiamo i campi utilizzando i nuovi oggetti Variable di Dedupe 3.0
fields = [
    dedupe.variables.String("manufacturer"),
    dedupe.variables.String("model"),
    dedupe.variables.Price("year"),
    dedupe.variables.Categorical(
        "transmission", 
        categories=['other', 'automatic', 'manual', 'a', 'cvt', 'm', 'dual clutch'],
        has_missing=True
    ),
    dedupe.variables.Categorical(
        "fuel_type", 
        categories=['gas', 'other', 'diesel', 'hybrid', 'electric', 'gasoline',
                   'biodiesel', 'flex fuel vehicle', 'compressed natural gas', 'propane'],
        has_missing=True
    ),
    dedupe.variables.Price("price"),
    dedupe.variables.Price("mileage", has_missing=True),
    dedupe.variables.String("location"),
    dedupe.variables.String("cylinders", has_missing=True),
    dedupe.variables.Categorical(
        "traction", 
        categories=['rwd', '4wd', 'fwd', 'awd', '4x2'], 
        has_missing=True
    ),
    dedupe.variables.Categorical(
        "body_type", 
        categories=['pickup', 'truck', 'other', 'coupe', 'suv', 'hatchback',
                   'minivan', 'sedan', 'offroad', 'van', 'convertible', 'wagon',
                   'bus', 'suv crossover', 'pickup truck'], 
        has_missing=True
    ),
    dedupe.variables.Categorical(
        "main_color", 
        categories=['white', 'blue', 'red', 'black', 'silver', 'grey', 'brown',
                   'yellow', 'orange', 'green', 'custom', 'purple', 'gray', 'unknown',
                   'gold', 'teal', 'pink'], 
        has_missing=True
    ),
    dedupe.variables.String("description", has_missing=True),
    dedupe.variables.Price("latitude", has_missing=True),
    dedupe.variables.Price("longitude", has_missing=True),
]

In [ ]:
# 1. Inizializza con Dedupe (per ricerca interna + linkage)
linker = dedupe.Dedupe(fields)

# 2. Prepara gli esempi dalla Ground Truth
matches = []
distinct = []

for _, row in gt_train.iterrows():
    # Recuperiamo i dati completi dai record usando gli ID della GT
    if row['id_A'] in data_train_only and row['id_B'] in data_train_only:
        pair = (data_train_only[row['id_A']], data_train_only[row['id_B']])
        if row['label'] == 1:
            matches.append(pair)
        else:
            distinct.append(pair)

# 3. Passa gli esempi
linker.mark_pairs({'match': matches, 'distinct': distinct})

# 4. PREPARE_TRAINING (Passando solo data_d una volta)
print("Preparazione dell'addestramento")
linker.prepare_training(
    data=data_train_only, 
    sample_size=15000, 
    blocked_proportion=0.5
) 

# 5. TRAIN
print("Addestramento Dedupe in corso")
linker.train()

Preparazione dell'addestramento
